In [2]:
import cv2
import numpy as np

img = cv2.imread('./images/donut1.png')
img_orig = img.copy()
cv2.imshow('Original Image',img)
cv2.waitKey()

img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.imshow('Gray Image',img_gray)
cv2.waitKey()

img_canny = cv2.Canny(img_gray,50,200)
cv2.imshow('Canny Edges',img_canny)
cv2.waitKey()

contours1,hierarchy1 = cv2.findContours(img_canny.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

img_black = np.zeros(img.shape,np.uint8)
img_black_orig = img_black.copy()

for c in contours1:
    accuracy = 0.0000003*cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c,accuracy,True)
    cv2.drawContours(img_black,[approx],0,(255,255,255),2)
    cv2.imshow('Contours Approx',img_black)
    cv2.waitKey()

kernel = np.ones((5,5),np.uint8)
img_dilate = cv2.dilate(img_black,kernel,iterations=2)
cv2.imshow('Dilated',img_dilate)
cv2.waitKey()

img_erode = cv2.erode(img_dilate,kernel,iterations=1)
cv2.imshow('Erode',img_erode)
cv2.waitKey()

img_gray = cv2.cvtColor(img_erode,cv2.COLOR_BGR2GRAY)
cv2.imshow('New Gray',img_gray)
cv2.waitKey()

contours2,hierarchy2 = cv2.findContours(img_gray.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
for c in contours2:
    cv2.drawContours(img_black_orig,[c],-1,(0,255,0),2)
    cv2.waitKey()
    cv2.imshow('NN',img_black_orig)
cv2.waitKey()
# print hierarchy2
new_contours=[]
new_hierarchy=[]
x=0
while x!=-1:
    # print('x ',x)
    a1=cv2.contourArea(contours2[x])
    # print('a1 ',a1)
    c1=hierarchy2[0][x][2]
    a2=0;
    if c1!=-1:
        a2=cv2.contourArea(contours2[c1])
        # print('a2 ',a2)
    # if (a1-a2)<=20:
    hierarchy2[0][x][2]=hierarchy2[0][c1][2]
# print(a1," ",a2)
    new_contours.append(contours2[x])
    new_hierarchy.append(hierarchy2[0][x].tolist())
    x=hierarchy2[0][x][0]
# print new_contours
# print hierarchy2
# print new_hierarchy
def sortalgo(ch):
    c,h=ch
    a1=cv2.contourArea(c)
    a2=0
    if h[2]!=-1:
        a2=cv2.contourArea(contours2[h[2]])
    # print (a1-a2)
    # print (a1," ",a2)
    return a1-a2
sc = [x for x,_ in sorted(zip(new_contours,new_hierarchy),key=sortalgo,reverse=False)]
hh = [y for _,y in sorted(zip(new_contours,new_hierarchy),key=sortalgo,reverse=False)]
# print hh
# print new_hierarchy
#     cv2.waitKey()
for (i,c) in enumerate(sc):
    cv2.drawContours(img_orig,[c],-1,(0,255,0),3)
    # print ("@ ",hh[i][2])
    if hh[i][2]!=-1:
        cv2.drawContours(img_orig,[contours2[hh[i][2]]],-1,(0,255,0),3)
    cv2.imshow('Contours sorted by area',img_orig)
    cv2.waitKey()
# for (i,c) in enumerate(new_contours):
#     cv2.drawContours(img_black_orig,[c],-1,(0,255,0),1)
#     M = cv2.moments(c)
#     if int(M['m00']==0):
#         print i+1
#     if int(M['m00'])!=0:
#         cx = int(M['m10']/M['m00'])
#         cy = int(M['m01']/M['m00'])
#         cv2.putText(img_black_orig,str(i+1),(cx,cy),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1)
#     cv2.waitKey()
#     cv2.imshow('New Contours',img_black_orig)
#     print (i)
cv2.destroyAllWindows()

